In [ ]:
import pandas as pd
import numpy as np
from math import cos, sin, sqrt, pi
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
_hierarchy_ = [
[['rat',         'mouse',      'squirrel'],         'rodentia',   'mammal'],
[['fruit bat',   'nectar bat'],                     'chiroptera', 'mammal'],
[['lemur',       'baboon',     'gorilla', 'human'], 'primate',    'mammal'],
[['frog'],                                          'anura',      'amphibian'],
[['salamander'],                                    'urodela',    'amphibian'],
[['rattlesnake', 'asp',        'cobra',   'viper'], 'squamata',   'reptile']
]
parent_lu = {}
order     = []
for row in _hierarchy_:
    order.extend(row[0])
    order.append(None)
    parent_lu[row[1]] = row[2]
    for animal in row[0]:
        parent_lu[animal] = row[1]

df = pd.DataFrame({'fm':['rat',       'mouse',      'squirrel', 'baboon',  'human', 'salamander',  'asp',   'viper'],
                   'to':['fruit bat', 'nectar bat', 'lemur',    'gorilla', 'frog',  'rattlesnake', 'cobra', 'cobra']})

cd  = rt.chordDiagram(df, [('fm','to')], equal_size_nodes=True, order=order,
                      w=384, h=384, x_ins=64, y_ins=64, 
                      draw_labels=True, label_style='circular', draw_background=False)

cd2 = rt.chordDiagram(df, [('fm','to')], equal_size_nodes=True, parent_lu=parent_lu,
                      w=384, h=384, x_ins=64, y_ins=64, 
                      draw_labels=True, label_style='circular', draw_background=False)

cd3 = rt.chordDiagram(df, [('fm','to')], equal_size_nodes=True, parent_lu=parent_lu,
                      w=384, h=384, x_ins=64, y_ins=64, draw_background=False)

rt.tile([cd,cd2,cd3])

In [ ]:
_svg_   = cd._repr_svg_()
_svg_b_ = _svg_[:_svg_.rindex('<')]
_svg_e_ = _svg_[_svg_.rindex('<'):]

a0,a1 = cd.node_to_arc['rattlesnake']
b0,b1 = cd.node_to_arc['viper']

to_rad = lambda d: d*pi/180

_extra_  = f'<line x1="{cd.cx + cd.r               * cos(to_rad(a0))}" y1="{cd.cy + cd.r               * sin(to_rad(a0))}" '
_extra_ += f'      x2="{cd.cx + (cd.r+4*cd.node_h) * cos(to_rad(a0))}" y2="{cd.cy + (cd.r+4*cd.node_h) * sin(to_rad(a0))}" stroke="#ff0000" stroke-width="0.5" />'

_extra_ += f'<line x1="{cd.cx + cd.r               * cos(to_rad(b1))}" y1="{cd.cy + cd.r               * sin(to_rad(b1))}" '
_extra_ += f'      x2="{cd.cx + (cd.r+4*cd.node_h) * cos(to_rad(b1))}" y2="{cd.cy + (cd.r+4*cd.node_h) * sin(to_rad(b1))}" stroke="#ff0000" stroke-width="0.5" />'

_extra_ += f'<path d="{cd.__genericArc__(a0, b1, cd.r+  cd.node_h, cd.r+2*cd.node_h)}" fill="#d0d0d0" stroke="#000000" />'
_extra_ += f'<path d="{cd.__genericArc__(a0, b1, cd.r+3*cd.node_h, cd.r+4*cd.node_h)}" fill="#c0c0c0" stroke="#000000" />'

rt.tile([_svg_b_ + _extra_ + _svg_e_])

In [ ]:
# COPIED INTO CHORD DIAGRAM MIXIN 2024-04-
# ... but was incorrrectly ordered
#
# orderedChildren() - return a list of children ordered by parent
# - lu is a lookup dictionary from child to parent
# - lu[child] = parent
# - returns a list of children ordered firstly by parent (only leaf children are included)
# -- None is used as a separator
# 
def orderedChildren(lu):
    children = {}
    for child in lu:
        if lu[child] not in children:
            children[lu[child]] = []
        children[lu[child]].append(child)
    ordered = []
    for parent in children:
        one_added = False
        for x in children[parent]:
            if x not in children:
                ordered.append(x)
                one_added = True
        if one_added:
            ordered.append(None)
    return ordered

orderedChildren(parent_lu)

In [ ]:
cd4 = rt.chordDiagram(df, [('fm','to')], equal_size_nodes=True, parent_lu=parent_lu,
                      w=512, h=512, x_ins=96, y_ins=96, node_h=16, txt_h=16, 
                      draw_labels=True, label_style='circular', draw_background=False)
svg_repr = cd4._repr_svg_()
svg0 = svg_repr[:svg_repr.rindex('<')]
svg1 = svg_repr[svg_repr.rindex('<'):]

svg_overlay = []
svg_overlay.append('<rect x="0" y="0" width="512" height="512" fill="#ffffff" fill-opacity="0.8" />')
def posInfo(_pos_, _color_='#000000'):
    x,y       = _pos_.xy()
    svg_overlay.append(f'<circle cx="{x}" cy="{y}" r="3" fill="{_color_}" />')
    for i in range(len(_pos_.attachmentPointVecs())):
        x,y,xv,yv = _pos_.attachmentPointVecs()[i]
        svg_overlay.append(f'<line x1="{x}" y1="{y}" x2="{x+20*xv}" y2="{y+20*yv}" stroke="{_color_}" />')
    svg_markup = _pos_.svg()
    svg_overlay.append(svg_markup[:5]+f' fill="none" stroke="{_color_}" '+svg_markup[5:])

position_infos = cd4.entityPositions('mammal')
for info in position_infos:
    posInfo(info)

position_infos = cd4.entityPositions('squamata')
for info in position_infos:
    posInfo(info)

position_infos = cd4.entityPositions('cobra')
for info in position_infos:
    posInfo(info)

rt.tile([svg0+''.join(svg_overlay)+svg1])

In [ ]:
# COPIED INTO CHORD DIAGRAM MIXIN 2024-04-06
# ... fixes the hierarchical issue...
child_lu, roots = {}, set()
for child in parent_lu:
    parent = parent_lu[child]
    if parent not in child_lu:
        child_lu[parent] = []
    child_lu[parent].append(child)
    if parent not in parent_lu:
        roots.add(parent)

def leafWalk(t, root):
    if root in t:
        _order_ = []
        for child in t[root]:
            _order_.extend(leafWalk(t,child))
        _order_.append(None)
        return _order_
    else:
        return [root]

order = []
for root in roots:
    suborder = leafWalk(child_lu, root)
    order.extend(suborder)
    order.append(None)

new_order, last_was_none = [], False
for x in order:
    if x is None:
        if not last_was_none:
            new_order.append(None)
        last_was_none = True
    else:
        new_order.append(x)
        last_was_none = False
        
new_order

In [ ]:
rt.addAnnotation(rt.entityAnnotation('reptile', description_str='As commonly defined, are a group of tetrapods with an ectothermic ("cold-blooded") metabolism and amniotic development.'))
rt.addAnnotation(rt.entityAnnotation('mammal',  description_str='Characterized by the presence of milk-producing mammary glands for feeding their young, a neocortex region of the brain, fur or hair, and three middle ear bones.'))
rt.annotateEntities(cd4, draw_text_border=True, include_description=True, max_line_w=160, txt_block_h_gap=10)

In [ ]:
rt.annotateEntities(cd4, draw_text_border=True, include_description=True, max_line_w=160, txt_block_v_gap=500, txt_block_h_gap=16)

In [ ]:
rt.annotateEntities(cd4, draw_text_border=True, include_description=True, max_line_w=160, txt_block_v_gap=500, txt_block_h_gap=10)